In [37]:
import numpy as np
import tensorflow as tf
session = tf.Session()

In [38]:
data_size = [10,10]
data_2d = np.random.normal(size = data_size) # se definen los datos como una matriz bidimensional de 10x10
x_data = tf.placeholder(shape=data_size, dtype=tf.float32) # se define el placeholder

#### en el caso de imagenes, tensorflow trabaja con 4 dimensiones, las cuales son:
      [cantidad de elementos, anchura, altura, canales de color]

# capa de convolucion
    paso 1: transforma el la matriz de 2d en una matriz de 4d
    paso 2: ejecuta la operacion de convolucion
    paso 3: realiza un squeeze para aplanar el resultado

In [39]:
def conv_layer_2d(input_2d, my_filter):
    input_3d = tf.expand_dims(input_2d, 0) # se añade 1 dimension
    input_4d = tf.expand_dims(input_3d, 3) # se añade 1 dimension
    convolution = tf.nn.conv2d(input_4d, filter=my_filter, strides=[1,2,2,1], padding="VALID") # se ejecuta la operaciond e convolucion
    output = tf.squeeze(convolution) # se quita las dimensiones añadidas anteriormente
    return(output)

In [40]:
my_filter = tf.Variable(tf.random_normal(shape=[2,2,1,1])) #[cantidad de elementos, anchura, altura, canales de color]
my_conv_output = conv_layer_2d(x_data, my_filter)

# capa de activacion

In [41]:
def activation(input_2d):
    return tf.nn.relu(input_2d)

In [42]:
my_activation_output = activation(my_conv_output)

# capa de maxpool
    -se encarga de eliminar valores mediocres
    -devuleve el valor maximo de los elementos que caen dentro de la ventana

In [43]:
def max_pool(input_2d, width, height): #"width" y "height" corresponden al tamaño de la ventana(en este caso 2x2)
    input_3d = tf.expand_dims(input_2d, 0) # se añade 1 dimension
    input_4d = tf.expand_dims(input_3d, 3) # se añade 1 dimension
    pooling = tf.nn.max_pool(input_4d, ksize=[1,height,width,1], strides=[1,1,1,1], padding="VALID")
    output = tf.squeeze(pooling) # se quita las dimensiones añadidas anteriormente
    return(output)

In [44]:
my_maxpool_output = max_pool(my_activation_output, width=2, height=2) # en este caso el tamaño de la ventana sera de 5

# capa de fullyconnected
    -conecta toda la informacion anterior y devuleve la salida
    -la salida se debe indicar, segun lo que necesitamos

In [45]:
def fully_connected(input_layer, num_output):
    flat_input = tf.reshape(input_layer, [-1]) # se aplanan los datos para convertirlos en vector para conocer el tamaño en la siguiente linea
    weight_shape = tf.squeeze(tf.stack([tf.shape(flat_input), [num_output]])) # se define el tamaño de la matriz de pesos
    weight = tf.random_normal(weight_shape, stddev=0.1) # matriz de pesos con el tamaño definido anteirormente y una desviacion estandar pequeña
    bias = tf.random_normal(shape=[num_output]) # se define el bias
    input_layer_2d = tf.expand_dims(flat_input, 0) # se expande la dimension del flat_input a 2d para poder multiplicar
    full_output = tf.add(tf.matmul(input_layer_2d, weight), bias) # operacion
    full_output_1d = tf.squeeze(full_output) #se quita las dimensiones extras
    return(full_output_1d)

In [46]:
my_full_output = fully_connected(my_maxpool_output, 5) # el valor de la salida en este caso sera 5

### hasta aqui la definicion de capas

# se inicializan variables 

In [47]:
init = tf.global_variables_initializer()
session.run(init)

# resultados de cada capa

In [48]:
# datos de entrada iniciales
data_2d

array([[ 1.5333449 ,  1.03816877,  0.68629536,  1.60640283,  1.75062124,
         0.45938764, -0.99924787, -0.36440907, -1.99117014, -0.68243987],
       [ 1.80204529, -1.6901531 , -0.09703476, -0.79127237, -1.34388878,
         0.56860244,  0.79385783, -1.28884097, -0.07109972, -0.44710002],
       [-0.6389813 , -1.72274238,  0.57758593,  0.83743969,  3.11546885,
        -1.23082476, -0.37205731, -0.04453746, -1.81024491,  0.02116152],
       [-0.75609847,  1.5427008 ,  0.3971531 ,  0.02143196, -0.54413901,
         1.46524972,  0.2612015 , -1.62054033,  0.71796075,  0.64971307],
       [ 2.06140683,  0.79814915,  0.17964037,  1.22963932,  0.00490042,
         0.17172208, -0.38832128, -1.16946669,  0.27857355, -2.8092286 ],
       [ 3.21141168,  1.25129168, -0.91466102,  1.17472666, -0.87946267,
        -1.14087736,  1.66263836, -0.65462736, -1.07399595, -0.44585839],
       [ 0.10623911,  0.97090575,  1.55771483, -0.1448572 , -1.83499486,
        -1.52241492,  0.40264154,  0.13755774

In [49]:
# el filtro
session.run(my_filter)

array([[[[ 0.5348398 ]],

        [[-0.15540694]]],


       [[[-0.27715585]],

        [[-0.08548772]]]], dtype=float32)

In [50]:
# capa de convolucion (multiplica el my_filter con los datos de data_1d)
print("INPUT: tamaño 10x10(correspondiente a data_2d)")
print("OPERACION: capa de convolucion con filtro de tamaño 2x2 + stride de tamaño 2x2")
print("SALIDA: tamaño 5x5")
print(session.run(my_conv_output, feed_dict={x_data: data_2d}))

INPUT: tamaño 10x10(correspondiente a data_2d)
OPERACION: capa de convolucion con filtro de tamaño 2x2 + stride de tamaño 2x2
SALIDA: tamaño 5x5
[[ 0.30379516  0.21194974  1.1887679  -0.5876481  -0.9009739 ]
 [ 0.00364864  0.06686654  1.8831058  -0.12592687 -1.2260091 ]
 [-0.01854705  0.05806331  0.31721336 -0.43079373  0.92134553]
 [-0.17123497  0.36275244 -1.1763558   0.50939655  0.37529916]
 [-0.253703    0.8579994   0.07948311 -0.27058303  0.00342241]]


In [52]:
# capa de activacion (elimina los valores negativos de la salida devuelta por la capa de convolucion)
print("INPUT: tamaño 5x5(correspondiente a la salida de la capa my_conv_output)")
print("OPERACION: se aplica funcion relu a los valores de la salida de la capa my_conv_output")
print("SALIDA: tamaño 5x5")
print(session.run(my_activation_output, feed_dict={x_data: data_2d}))

INPUT: tamaño 5x5(correspondiente a la salida de la capa my_conv_output)
OPERACION: se aplica funcion relu a los valores de la salida de la capa my_conv_output
SALIDA: tamaño 5x5
[[ 0.30379516  0.21194974  1.1887679  -0.         -0.        ]
 [ 0.00364864  0.06686654  1.8831058  -0.         -0.        ]
 [-0.          0.05806331  0.31721336 -0.          0.92134553]
 [-0.          0.36275244 -0.          0.50939655  0.37529916]
 [-0.          0.8579994   0.07948311 -0.          0.00342241]]


In [54]:
# capa de maxpool(toma en fila valores de 5 en 5 y elije el mas grande)
print("INPUT: tamaño 5x5(correspondiente a la salida de la capa de activacion)")
print("OPERACION: maxpooling con ventana de tamaño 2x2 + stride de tamaño 1")
print("SALIDA: tamaño 4x4")
print(session.run(my_maxpool_output, feed_dict={x_data: data_2d}))

INPUT: tamaño 5x5(correspondiente a la salida de la capa de activacion)
OPERACION: maxpooling con ventana de tamaño 2x2 + stride de tamaño 1
SALIDA: tamaño 4x4
[[ 0.30379516  1.8831058   1.8831058  -0.        ]
 [ 0.06686654  1.8831058   1.8831058   0.92134553]
 [ 0.36275244  0.36275244  0.50939655  0.92134553]
 [ 0.8579994   0.8579994   0.50939655  0.50939655]]


In [55]:
# capa fully_conected (conecta toda la informacion anterior y devuleve la salida)
print("INPUT: tamaño 4x4(correspondiente a la salida de la capa de max_pool)")
print("OPERACION: conectar la entrada data_2d con los 5 valores de salida")
print("SALIDA: tamaño 5")
print(session.run(my_full_output, feed_dict={x_data: data_1d}))

INPUT: tamaño 4x4(correspondiente a la salida de la capa de max_pool)
OPERACION: conectar la entrada data_2d con los 5 valores de salida
SALIDA: tamaño 5
[-1.2846904  -0.24092372 -0.6650247  -1.9412917   0.12668753]
